<a href="https://colab.research.google.com/github/NicoPatalagua/BigData/blob/master/window_ex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from pyspark.sql import functions as f
from pyspark.sql import SparkSession

spark =  SparkSession.builder.appName("test").getOrCreate()

In [0]:
df_ec = spark.read.csv("EC.csv", header=True, inferSchema=True)
df_avh = spark.read.csv("AVH.csv", header=True, inferSchema=True)
df_aval = spark.read.csv("AVAL.csv", header=True, inferSchema=True)

In [0]:
df = df_ec.select(f.lit("ECOPETROL").alias("Empresa"), "*")
df = df.union(df_avh.select(f.lit("AVIANCA").alias("Empresa"), "*"))
df = df.union(df_aval.select(f.lit("AVAL").alias("Empresa"), "*"))

In [0]:
df.groupBy("Empresa").count().show()

+---------+-----+
|  Empresa|count|
+---------+-----+
|     AVAL|  253|
|ECOPETROL|  253|
|  AVIANCA|  253|
+---------+-----+



In [0]:
from pyspark.sql import Window

w  = Window.partitionBy("Empresa").orderBy(df.Close.desc())

In [0]:
df.select("Empresa", "Date", "Close", f.row_number().over(w).alias("nrow")) \
.where("nrow<=5").show()

+---------+-------------------+---------+----+
|  Empresa|               Date|    Close|nrow|
+---------+-------------------+---------+----+
|     AVAL|2020-01-14 00:00:00|     8.88|   1|
|     AVAL|2020-01-08 00:00:00|     8.86|   2|
|     AVAL|2020-01-09 00:00:00|     8.85|   3|
|     AVAL|2020-01-13 00:00:00|     8.85|   4|
|     AVAL|2020-01-06 00:00:00|     8.84|   5|
|ECOPETROL|2019-04-08 00:00:00|21.969999|   1|
|ECOPETROL|2019-04-01 00:00:00|21.879999|   2|
|ECOPETROL|2019-04-10 00:00:00|21.860001|   3|
|ECOPETROL|2019-04-02 00:00:00|    21.84|   4|
|ECOPETROL|2019-04-03 00:00:00|    21.83|   5|
|  AVIANCA|2020-02-19 00:00:00|      5.0|   1|
|  AVIANCA|2020-02-20 00:00:00|      5.0|   2|
|  AVIANCA|2020-02-12 00:00:00|     4.94|   3|
|  AVIANCA|2020-02-04 00:00:00|     4.92|   4|
|  AVIANCA|2020-02-05 00:00:00|     4.92|   5|
+---------+-------------------+---------+----+



In [0]:
df.select("Empresa", "Date", "Close", f.max("Close").over(w).alias("First"))\
.select("Empresa", "First").distinct().show()

+---------+---------+
|  Empresa|    First|
+---------+---------+
|     AVAL|     8.88|
|ECOPETROL|21.969999|
|  AVIANCA|      5.0|
+---------+---------+



In [0]:
w  = Window.partitionBy("Empresa").orderBy("Date").rowsBetween(-3, 0)

In [0]:
df.select("Empresa", "Date", f.sum("Close").over(w).alias("anterior")).show(10, False)

+-------+-------------------+------------------+
|Empresa|Date               |anterior          |
+-------+-------------------+------------------+
|AVAL   |2019-04-10 00:00:00|8.42              |
|AVAL   |2019-04-11 00:00:00|16.82             |
|AVAL   |2019-04-12 00:00:00|25.16             |
|AVAL   |2019-04-15 00:00:00|33.42             |
|AVAL   |2019-04-16 00:00:00|33.22             |
|AVAL   |2019-04-17 00:00:00|33.15             |
|AVAL   |2019-04-18 00:00:00|33.050000000000004|
|AVAL   |2019-04-22 00:00:00|33.01             |
|AVAL   |2019-04-23 00:00:00|33.03             |
|AVAL   |2019-04-24 00:00:00|32.760000000000005|
+-------+-------------------+------------------+
only showing top 10 rows

